In [1]:
import pandas as pd
import numpy as np
import time
import os

# # set PYWTK_CACHE_DIR to locate WIND Toolkit data
# # will download from AWS as needed
os.environ["PYWTK_CACHE_DIR"] = os.path.join(os.environ["HOME"], "pywtk-data")

from powerscenarios.parser import Parser
from powerscenarios.grid_copy import Grid
# from powerscenarios.grid import Grid

# show multiple cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option("display.max_rows", 20)
pd.set_option("display.max_columns", 100)

# plotting (optional)
import cufflinks as cl

cl.go_offline()


# or parse RTS grid .csv files

In [2]:
# RTS-GMLC grid
# https://github.com/GridMod/RTS-GMLC
grid_name = "RTS"

data_dir = "../data/grid-data"

bus_csv_filename = os.path.join(data_dir, grid_name, "bus.csv")
gen_csv_filename = os.path.join(data_dir, grid_name, "gen.csv")

parser = Parser()

# if solar2wind, will replace all solar to wind
bus_df, gen_df, wind_gen_df = parser.parse_rts_csvs(
    bus_csv_filename, gen_csv_filename, solar2wind=True
)

# # see what you got
# print("bus_df:")
# bus_df.head()
# print("gen_df:")
# gen_df.head()
# print("wind_gen_df:")
# wind_gen_df.head()

## instantiate Grid class

In [3]:
# to instantiate a grid we need: name, bus, generator, and wind generator dataframes from Parser
# really, we only wind generators, will change in the future
grid_name= "RTS"
grid = Grid(grid_name, bus_df, gen_df, wind_gen_df)
grid
print(grid.info())

Grid(name=RTS, buses=73, generators=158, wind_generators=61, wind_sites=0)


RTS grid info: 

 number of buses: 73
 number of generators: 158
 number of wind generators: 61
 number of solar generators: 0
 total generator capacity: 14549.80 MW
 wind capacity/penetration: 5423.80 MW / 37.28%
 solar capacity/penetration: 0.00 MW / 0.00%


In [4]:
# retrieve wind sites matching current wind penetration

# retrieve wind sites (wind_sites are initially set to empty df )
# grid
grid.retrieve_wind_sites(method="simple proximity")
# grid
# grid.wind_sites.head()

Retrieving wind sites ...
Done


## make tables  
if not available, will download data from AWS as needed

In [5]:
grid.make_tables(
    actuals_start=pd.Timestamp("2007-07-01 00:00:00", tz="utc"),
    actuals_end=pd.Timestamp("2007-7-31 23:55:00", tz="utc"),
    scenarios_start=pd.Timestamp("2008-07-01 00:00:00", tz="utc"),
    # scenarios_end=pd.Timestamp("2013-12-31 23:55:00", tz="utc"),
    scenarios_end=pd.Timestamp("2008-07-31 23:55:00", tz="utc"),
)

Retrieving WTK data ...
Done
Retrieving WTK data ...
Done


## change actuals year if needed

In [6]:
# for actuals, make year you want
grid.actuals.index = grid.actuals.index.map(lambda t: t.replace(year=2020))
# see what you got
print("\nactuals_df:")
grid.actuals.head()
print("\nscenarios_df:")
grid.scenarios.head()


actuals_df:


,320_PV_1,314_PV_1,314_PV_2,313_PV_1,314_PV_3,314_PV_4,313_PV_2,310_PV_1,324_PV_1,312_PV_1,310_PV_2,324_PV_2,324_PV_3,113_PV_1,319_PV_1,215_PV_1,102_PV_1,101_PV_1,102_PV_2,104_PV_1,212_CSP_1,101_PV_2,101_PV_3,101_PV_4,103_PV_1,119_PV_1,308_RTPV_1,313_RTPV_1,313_RTPV_2,313_RTPV_3,313_RTPV_4,313_RTPV_5,313_RTPV_6,313_RTPV_7,313_RTPV_8,313_RTPV_9,313_RTPV_10,313_RTPV_11,313_RTPV_12,320_RTPV_1,320_RTPV_2,320_RTPV_3,313_RTPV_13,320_RTPV_4,320_RTPV_5,118_RTPV_1,118_RTPV_2,118_RTPV_3,118_RTPV_4,118_RTPV_5,118_RTPV_6,320_RTPV_6,118_RTPV_7,118_RTPV_8,118_RTPV_9,118_RTPV_10,213_RTPV_1,309_WIND_1,317_WIND_1,303_WIND_1,122_WIND_1,TotalPower
IssueTime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-07-01 00:00:00+00:00,16.218550,41.414414,38.797422,24.511422,84.779132,51.6,30.647079,45.647396,49.7,69.013369,14.417227,51.6,51.0,6.975087,121.929042,30.432878,5.284509,5.123211,5.197474,3.553900,53.492969,5.013699,5.044731,5.557238,4.435381,0.005244,11.677268,37.729197,23.278332,23.397493,26.240505,29.448352,26.835649,22.121362,28.620972,30.697347,40.932191,20.303027,12.218975,9.617362,12.504782,8.967077,7.387033,10.693017,7.450221,1.120939,1.896862,3.009746,2.222464,0.733639,0.599265,5.024406,0.612119,0.614713,1.206295,0.527817,9.493373,132.237949,556.944189,847.0,180.424557,2955.179867
2020-07-01 00:05:00+00:00,16.767048,42.581107,39.202159,24.521047,85.156181,51.6,30.680218,45.632936,49.7,69.009940,14.486906,51.6,51.0,6.393500,125.138758,29.192745,5.264570,5.142888,5.254364,3.331152,53.934090,4.952212,5.091283,5.475201,4.403251,0.008594,11.264889,37.603480,23.429308,23.264851,25.868566,29.651657,26.713799,21.820592,28.268743,29.904143,40.168640,19.999802,11.961589,9.893194,12.914965,9.122319,7.209290,10.883749,7.590451,1.248197,1.903111,2.973869,2.038320,0.664045,0.565748,5.071882,0.572054,0.566228,1.094155,0.443573,7.114726,132.885411,572.269792,847.0,185.593991,2975.059280
2020-07-01 00:10:00+00:00,17.352792,43.497360,39.397081,24.688698,85.567714,51.6,31.277219,45.857518,49.7,69.310057,14.908601,51.6,51.0,5.853451,129.823280,27.634051,5.178375,5.084149,5.218333,3.014879,55.262720,4.864652,5.085984,5.387610,4.345550,0.009781,10.951315,37.520525,23.649648,23.192484,25.195216,30.147130,26.452134,21.607616,28.229024,29.133974,39.492254,20.074452,12.060521,10.158056,13.343381,9.273175,7.031967,11.082451,7.815938,1.415534,2.014298,2.984024,1.951773,0.662027,0.560171,5.319582,0.519178,0.490343,0.962967,0.376181,5.327745,133.614397,588.296443,847.0,189.263270,2999.689048
2020-07-01 00:15:00+00:00,17.772730,43.187182,39.367452,25.308155,86.221077,51.6,32.138778,46.113121,49.7,69.804438,15.592334,51.6,51.0,5.506304,133.992470,26.784092,5.082690,5.045664,5.182326,2.690151,56.734059,4.802985,5.060221,5.325876,4.355086,0.008016,10.433651,37.578887,23.775425,23.222061,24.560665,30.708072,26.324115,21.580692,28.313068,28.493793,38.990402,19.985708,12.063366,10.346116,13.567343,9.337239,6.965034,11.112188,7.952829,1.492059,2.039280,3.081292,1.776888,0.677965,0.601104,5.567361,0.467886,0.401782,0.835430,0.329836,5.418804,134.115394,601.258549,847.0,190.950487,3021.299979
2020-07-01 00:20:00+00:00,18.173115,42.005930,39.317284,25.846124,87.585799,51.6,32.658785,46.441820,49.7,70.584327,16.227496,51.6,51.0,5.355208,137.046729,26.762298,5.089029,5.091152,5.208223,2.408837,57.589272,4.870098,5.147959,5.387816,4.340262,0.001517,9.387169,37.710131,23.690756,23.146248,24.197482,31.028032,26.340611,21.555903,28.321478,28.174631,38.874231,19.858998,11.980738,10.478560,13.800619,9.329806,7.077890,11.057630,8.085764,1.419466,1.887443,3.219483,1.482678,0.683080,0.664414,5.708483,0.437396,0.353257,0.799007,0.348745,6.203544,135.227869,615.587441,847.0,191.486970,3043.645031



scenarios_df:


,320_PV_1,314_PV_1,314_PV_2,313_PV_1,314_PV_3,314_PV_4,313_PV_2,310_PV_1,324_PV_1,312_PV_1,310_PV_2,324_PV_2,324_PV_3,113_PV_1,319_PV_1,215_PV_1,102_PV_1,101_PV_1,102_PV_2,104_PV_1,212_CSP_1,101_PV_2,101_PV_3,101_PV_4,103_PV_1,119_PV_1,308_RTPV_1,313_RTPV_1,313_RTPV_2,313_RTPV_3,313_RTPV_4,313_RTPV_5,313_RTPV_6,313_RTPV_7,313_RTPV_8,313_RTPV_9,313_RTPV_10,313_RTPV_11,313_RTPV_12,320_RTPV_1,320_RTPV_2,320_RTPV_3,313_RTPV_13,320_RTPV_4,320_RTPV_5,118_RTPV_1,118_RTPV_2,118_RTPV_3,118_RTPV_4,118_RTPV_5,118_RTPV_6,320_RTPV_6,118_RTPV_7,118_RTPV_8,118_RTPV_9,118_RTPV_10,213_RTPV_1,309_WIND_1,317_WIND_1,303_WIND_1,122_WIND_1,TotalPower,Deviation
IssueTime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2008-07-01 00:00:00+00:00,0.715218,0.270804,0.278728,-0.567792,0.749133,0.302183,-0.991153,-0.510286,1.850688,-0.755227,-0.392377,2.031753,2.503366,0.312774,4.471250,0.252137,0.0,0.0,0.0,0.000951,1.196545,-0.008951,0.0,0.0,0.007094,2.774768,-0.860116,-0.574876,0.006230,-0.242285,-0.238137,-0.069972,-0.270488,-0.181937,-0.388462,0.176246,-0.105225,-0.831587,0.390182,0.336674,0.0,0.287751,-0.383931,0.293718,0.452162,0.164168,0.369925,0.128139,0.358193,0.244219,0.358118,0.0,0.318750,0.723607,0.0,0.0,-0.472592,-0.507922,12.198853,12.722237,4.216704,2549.204979,43.109951
2008-07-01 00:05:00+00:00,0.398438,0.642317,0.479452,-0.647758,1.049654,0.675211,-1.821015,-0.500740,3.506852,-0.839063,-0.591158,2.162318,2.845631,0.428032,6.108293,0.465416,0.0,0.0,0.0,0.002148,1.341834,0.000000,0.0,0.0,0.004724,1.110492,-0.380126,-1.023936,-0.169553,-0.177893,-0.067646,-0.151333,-0.261106,-0.212115,-0.567463,0.395224,-0.112157,-0.679732,0.118237,0.171924,0.0,0.201941,-0.484215,0.129280,0.377276,0.179408,0.541502,0.107540,0.626671,0.341929,0.578564,0.0,0.377553,0.732218,0.0,0.0,-0.234934,-0.506389,7.801384,14.820621,1.146595,2592.314930,40.440346
2008-07-01 00:10:00+00:00,0.155444,0.196970,0.321208,-0.490707,1.231008,0.804731,-1.609841,-0.445234,5.436855,-0.797381,-0.610940,1.810420,1.623449,0.339083,5.990376,0.535527,0.0,0.0,0.0,0.006699,1.238475,0.000000,0.0,0.0,0.005738,0.264303,-0.552985,-0.993197,-0.280301,-0.283915,0.070766,0.072109,-0.019967,-0.374138,-0.561291,0.389375,-0.267869,-0.764635,-0.111423,0.064161,0.0,0.066000,-0.542709,-0.072439,0.264691,0.384571,0.728797,0.127840,0.837691,0.395980,0.714943,0.0,0.484485,0.731564,0.0,0.0,-0.196102,-0.019816,13.422890,12.799867,2.510377,2632.755276,45.031505
2008-07-01 00:15:00+00:00,0.082397,0.185575,0.205926,-0.452676,1.263241,0.711482,-0.903907,-0.149850,5.919570,-0.403514,-0.171492,1.689490,1.237216,0.289391,6.360892,0.508279,0.0,0.0,0.0,0.011250,0.955359,0.000000,0.0,0.0,0.029236,-0.193630,-0.535465,-0.503665,-0.032150,-0.132313,0.132322,0.501624,0.250102,-0.344417,-0.267288,0.457168,-0.237755,-0.816512,-0.119631,0.050019,0.0,-0.019458,-0.443219,-0.037359,0.127512,0.445384,0.732976,0.035661,0.447040,0.457495,0.810812,0.0,0.652088,0.906013,0.0,0.0,-0.314467,0.154794,15.368009,11.621293,7.151284,2677.786781,53.672134
2008-07-01 00:20:00+00:00,-0.225248,0.340896,0.332890,-0.139170,1.252326,0.611133,0.006406,-0.035232,4.407230,-0.395548,-0.011725,1.654865,0.911689,0.253366,6.506395,1.026376,0.0,0.0,0.0,0.007767,0.511131,0.000000,0.0,0.0,0.023411,0.371794,-0.678838,0.539007,0.402553,0.176899,0.582561,0.776561,0.551342,-0.150993,-0.043242,0.600976,-0.331154,-0.903871,-0.227552,-0.114421,0.0,-0.130909,-0.330502,-0.101327,0.061591,0.371254,0.688949,-0.072003,0.000000,0.441422,0.683438,0.0,0.677851,0.924263,0.0,0.0,-0.234794,-0.275672,11.154333,9.021268,11.292881,2731.458915,52.762621


## generate_wind_scenarios

In [7]:
sim_timestamps = [pd.Timestamp("2020-07-01 00:15:00+0000", tz="UTC"),] 


# other parameters
# sampling_method="monte carlo"
sampling_method="importance"
n_scenarios = 1
n_periods = 1

########################################################

all_weights_df = pd.DataFrame(index=sim_timestamps,columns=range(1,n_scenarios+1))

# create multiindex df for all generated scenarios
# three arrays for multiindex:
a1 = [x for x in sim_timestamps for k in range(n_scenarios * n_periods)]
a2 = [x for x in range(1, n_scenarios + 1) for k in range(n_periods)] * len(
    sim_timestamps
)
a3 = [
    t + pd.Timedelta("5min") * k
    for t in sim_timestamps
    for k in list(range(n_periods)) * n_scenarios
]

index = pd.MultiIndex.from_arrays([a1,a2,a3],names=['sim_timestamp','scenario_nr','period_timestamp'])
all_scenarios_df = pd.DataFrame(index=index,columns=grid.wind_generators["GenUID"].values)



for sim_timestamp in sim_timestamps:
    #print("sim_timestamp = {}".format(sim_timestamp))
    # random_seed = np.random.randint(2 ** 31 - 1)
    random_seed = 594081473
    #print("random_seed = {}".format(random_seed))
    scenarios_df, weights_df = grid.generate_wind_scenarios(
        sim_timestamp,
        power_quantiles=[0.0, 0.1, 0.9, 1.0],
        sampling_method=sampling_method,
        n_scenarios=n_scenarios,
        n_periods=n_periods,
        #random_seed=6,
        random_seed=random_seed,
        output_format=0,
    )
    #all_scenarios_df=pd.concat([all_scenarios_df,scenarios_df])
    all_scenarios_df.loc[sim_timestamp]=scenarios_df

    #all_weights_df=pd.concat([all_weights_df,weights_df])
    all_weights_df.loc[sim_timestamp]=weights_df.loc[sim_timestamp]



# copy wanted actuals
all_actuals_df=grid.actuals.loc[sim_timestamps].drop("TotalPower", axis=1).copy()
# match index name to all_scenarios_df
all_actuals_df.index.name = "sim_timestamp"

print('\nall_actuals_df:')
all_actuals_df
print('\nall_scenarios_df:')
all_scenarios_df
print('\nall_weights_df:')
all_weights_df





all_actuals_df:


,320_PV_1,314_PV_1,314_PV_2,313_PV_1,314_PV_3,314_PV_4,313_PV_2,310_PV_1,324_PV_1,312_PV_1,310_PV_2,324_PV_2,324_PV_3,113_PV_1,319_PV_1,215_PV_1,102_PV_1,101_PV_1,102_PV_2,104_PV_1,212_CSP_1,101_PV_2,101_PV_3,101_PV_4,103_PV_1,119_PV_1,308_RTPV_1,313_RTPV_1,313_RTPV_2,313_RTPV_3,313_RTPV_4,313_RTPV_5,313_RTPV_6,313_RTPV_7,313_RTPV_8,313_RTPV_9,313_RTPV_10,313_RTPV_11,313_RTPV_12,320_RTPV_1,320_RTPV_2,320_RTPV_3,313_RTPV_13,320_RTPV_4,320_RTPV_5,118_RTPV_1,118_RTPV_2,118_RTPV_3,118_RTPV_4,118_RTPV_5,118_RTPV_6,320_RTPV_6,118_RTPV_7,118_RTPV_8,118_RTPV_9,118_RTPV_10,213_RTPV_1,309_WIND_1,317_WIND_1,303_WIND_1,122_WIND_1
sim_timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-07-01 00:15:00+00:00,17.77273,43.187182,39.367452,25.308155,86.221077,51.6,32.138778,46.113121,49.7,69.804438,15.592334,51.6,51.0,5.506304,133.99247,26.784092,5.08269,5.045664,5.182326,2.690151,56.734059,4.802985,5.060221,5.325876,4.355086,0.008016,10.433651,37.578887,23.775425,23.222061,24.560665,30.708072,26.324115,21.580692,28.313068,28.493793,38.990402,19.985708,12.063366,10.346116,13.567343,9.337239,6.965034,11.112188,7.952829,1.492059,2.03928,3.081292,1.776888,0.677965,0.601104,5.567361,0.467886,0.401782,0.83543,0.329836,5.418804,134.115394,601.258549,847.0,190.950487



all_scenarios_df:


,,,320_PV_1,314_PV_1,314_PV_2,313_PV_1,314_PV_3,314_PV_4,313_PV_2,310_PV_1,324_PV_1,312_PV_1,310_PV_2,324_PV_2,324_PV_3,113_PV_1,319_PV_1,215_PV_1,102_PV_1,101_PV_1,102_PV_2,104_PV_1,212_CSP_1,101_PV_2,101_PV_3,101_PV_4,103_PV_1,119_PV_1,308_RTPV_1,313_RTPV_1,313_RTPV_2,313_RTPV_3,313_RTPV_4,313_RTPV_5,313_RTPV_6,313_RTPV_7,313_RTPV_8,313_RTPV_9,313_RTPV_10,313_RTPV_11,313_RTPV_12,320_RTPV_1,320_RTPV_2,320_RTPV_3,313_RTPV_13,320_RTPV_4,320_RTPV_5,118_RTPV_1,118_RTPV_2,118_RTPV_3,118_RTPV_4,118_RTPV_5,118_RTPV_6,320_RTPV_6,118_RTPV_7,118_RTPV_8,118_RTPV_9,118_RTPV_10,213_RTPV_1,309_WIND_1,317_WIND_1,303_WIND_1,122_WIND_1
sim_timestamp,scenario_nr,period_timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-07-01 00:15:00+00:00,1,2020-07-01 00:15:00+00:00,17.451487,43.141237,39.541156,24.145483,84.738072,50.721818,31.573315,45.375874,49.7,68.128817,13.941281,51.6,51.0,4.849324,129.501058,27.248233,5.192582,4.966827,5.009372,2.920741,55.317546,4.864419,4.463819,5.273088,4.294739,0.0,10.951315,37.703706,23.829392,22.895853,25.126485,30.332484,26.196709,21.692872,27.999424,28.004873,38.111976,20.124969,12.116814,10.251837,13.254653,9.25453,6.797179,11.139835,7.752009,1.686084,2.014298,2.984024,1.951773,0.632467,0.505473,5.345339,0.51681,0.365573,1.016796,0.371148,5.216287,132.902746,587.288029,839.205073,188.513937



all_weights_df:


,1
2020-07-01 00:15:00+00:00,0.638725
